# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Running through an example

In [2]:
# EW vulnerable.
vuln_ns, vuln_ew = False, True
# you sit North and hold:
hand = 'QT5.T8.KQT87.864'
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 3, False)

# the auction goes:
auction = ["PAD_START","PAD_START","PAD_START","PASS"]
bid = bot_bid.bid(auction)

auction = ["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"]
# what's your bid?
bid = bot_bid.bid(auction)


In [3]:
bid.samples

['QTx.T8.KQT8x.8xx AKxx.AKQx.J9x.xx xxx.J9x.Ax.AKQT9 J98.xxxx.xxx.Jxx 0.88559 \n P-P-1N-2C-P-2S-P-P-P (6.0, 7.0)  \n P-P-1N-2C-P-P-P (8.3, 8.5)  \n P-P-1N-2C-P-2H-P-P-P (5.5, 7.2)  \n P-P-1N-2C-P-2D-P-P-X-P-2H-3D-X-P-P-P (8.3, 7.9) ',
 'QTx.T8.KQT8x.8xx AKxx.Qxxx.Ax.AT9 Jxx.AK9.9x.KQJxx 98x.Jxxx.Jxxx.xx 0.86874 \n P-P-1N-2C-P-2S-P-P-P (6.7, 7.8)  \n P-P-1N-2C-P-P-P (8.3, 8.1)  \n P-P-1N-2C-P-2H-P-P-P (5.5, 7.6)  \n P-P-1N-2C-P-2D-P-P-X-P-2H-3D-X-P-P-P (7.7, 8.2) ',
 'QTx.T8.KQT8x.8xx KJx.Axx.Axx.KQTx A98xx.KQ9xx.x.J9 xx.Jxx.J9xx.Axxx 0.86392 \n P-P-1N-2C-P-2S-P-P-P (7.9, 7.9)  \n P-P-1N-2C-P-P-P (4.3, 5.6)  \n P-P-1N-2C-P-2H-P-P-P (6.8, 7.5)  \n P-P-1N-2C-P-2D-P-P-P (5.9, 7.0) ',
 'QTx.T8.KQT8x.8xx Axx.AKJx.AJ9x.xx K98.9xx.xx.AKQ9x Jxxx.Qxxx.xx.JTx 0.85282 \n P-P-1N-2C-P-2S-P-P-P (5.5, 6.3)  \n P-P-1N-2C-P-P-P (8.1, 7.4)  \n P-P-1N-2C-P-2H-P-P-P (4.7, 6.2)  \n P-P-1N-2C-P-2D-P-P-X-P-2H-3D-X-P-P-P (7.0, 6.5) ',
 'QTx.T8.KQT8x.8xx xxx.AQx.AJ9x.KJx AKJxx.KJxxx.x.Qx 98.9xx.xxx.AT9xx 0.8256

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
QTx.T8.KQT8x.8xx K8xx.AQ9x.A9x.Ax A9x.Kxx.xx.KQT9x Jxx.Jxxx.Jxx.Jxx
QTx.T8.KQT8x.8xx Ax.A9x.A9xx.AJTx KJ9xx.KQxxx.J.xx 8xx.Jxx.xxx.KQ9x
QTx.T8.KQT8x.8xx K9xx.AQxx.A9x.Kx A8x.KJ.xxx.AQTxx Jxx.9xxxx.Jx.J9x
QTx.T8.KQT8x.8xx AJ.Jxx.A9xx.AKTx K98xx.AK9xx..Jxx xxx.Qxx.Jxxx.Q9x
QTx.T8.KQT8x.8xx Jxx.AK9x.Axx.AJx AKx.QJx.Jx.KT9xx 98xx.xxxx.9xx.Qx
QTx.T8.KQT8x.8xx Kx.Axx.AJxx.A9xx A8xxx.KQJ9x..KQT J9x.xxx.9xxx.Jxx
QTx.T8.KQT8x.8xx AK9x.A9x.Axx.QTx 8xx.KQxx.J.AKxxx Jxx.Jxxx.9xxx.J9
QTx.T8.KQT8x.8xx 8xx.AQx.Axx.KQJT AKJxx.KJ9xx.x.xx 9x.xxx.J9xx.A9xx
QTx.T8.KQT8x.8xx xxx.QJx.Ax.AKQT9 AK9x.K9xxx.9x.Jx J8x.Axx.Jxxx.xxx
QTx.T8.KQT8x.8xx A8xx.AKQx.Ax.Txx K9x.9xx.Jx.AK9xx Jxx.Jxxx.9xxx.QJ
QTx.T8.KQT8x.8xx J98x.Axx.AJxx.AK AKx.QJxx.9.Qxxxx xxx.K9xx.xxx.JT9
QTx.T8.KQT8x.8xx Axxx.AKJx.AJx.xx K8x.Qxxx.x.AJT9x J9x.9xx.9xxx.KQx
QTx.T8.KQT8x.8xx AKxx.Kxxx.Axx.Jx 9xx.AJxx.x.AKT9x J8x.Q9x.J9xx.Qxx
QTx.T8.KQT8x.8xx AK9x.Jxx.Axx.Axx J8x.AKxx.x.KQJ9x xxx.Q9xx.J9xx.Tx
QTx.T8.KQT8x.8xx K9x.AKx.J9xx